# Network Sniffing
- sniffing packets in unswitched network is fairly easy as packets are usually broadcasted to all the hosts in the network
- hub is a network device that broadcasts all the packets e.g. and simply running the interface card in promiscuous mode would collect all the data that was broadcasted
- sniffing packets in switched network is a bit challange
- if systems are behind router or network switch they're using swtiched network
    - packets are redirected to destination based on their MAC address

In [6]:
! echo kali | sudo -S ifconfig

[sudo] password for kali: eth0: flags=4163<UP,BROADCAST,RUNNING,MULTICAST>  mtu 1500
        inet 192.168.195.161  netmask 255.255.255.0  broadcast 192.168.195.255
        inet6 fe80::20c:29ff:fe17:1a37  prefixlen 64  scopeid 0x20<link>
        ether 00:0c:29:17:1a:37  txqueuelen 1000  (Ethernet)
        RX packets 572645  bytes 714907354 (681.7 MiB)
        RX errors 0  dropped 0  overruns 0  frame 0
        TX packets 249855  bytes 19674060 (18.7 MiB)
        TX errors 0  dropped 0 overruns 0  carrier 0  collisions 0

lo: flags=73<UP,LOOPBACK,RUNNING>  mtu 65536
        inet 127.0.0.1  netmask 255.0.0.0
        inet6 ::1  prefixlen 128  scopeid 0x10<host>
        loop  txqueuelen 1000  (Local Loopback)
        RX packets 374981  bytes 138144709 (131.7 MiB)
        RX errors 0  dropped 0  overruns 0  frame 0
        TX packets 374981  bytes 138144709 (131.7 MiB)
        TX errors 0  dropped 0 overruns 0  carrier 0  collisions 0



In [7]:
#ifconfig <interfaceCard> promisc on
! echo kali | sudo -S ifconfig eth0 promisc

[sudo] password for kali: 

In [5]:
# trun off promisc mode
! echo kali | sudo -S ifconfig eth0 -promisc

[sudo] password for kali: 

In [8]:
#ifconfig <interfaceCard>
! echo kali | sudo -S ifconfig eth0

[sudo] password for kali: eth0: flags=4419<UP,BROADCAST,RUNNING,PROMISC,MULTICAST>  mtu 1500
        inet 192.168.195.161  netmask 255.255.255.0  broadcast 192.168.195.255
        inet6 fe80::20c:29ff:fe17:1a37  prefixlen 64  scopeid 0x20<link>
        ether 00:0c:29:17:1a:37  txqueuelen 1000  (Ethernet)
        RX packets 572645  bytes 714907354 (681.7 MiB)
        RX errors 0  dropped 0  overruns 0  frame 0
        TX packets 249855  bytes 19674060 (18.7 MiB)
        TX errors 0  dropped 0 overruns 0  carrier 0  collisions 0



## Install vsftpd server

- install ftpd server for demo - remember FTP is not encrypted; typically use OpenSSH
- detail instruction: http://itsfoss.com/set-ftp-server-linux


In [9]:
! echo kali | sudo -S apt install vsftpd -y

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following packages were automatically installed and are no longer required:
  libomp-9-dev libomp5-9 linux-image-5.9.0-kali4-amd64 python3-plotly
Use 'sudo apt autoremove' to remove them.
The following NEW packages will be installed:
  vsftpd
0 upgraded, 1 newly installed, 0 to remove and 535 not upgraded.
Need to get 155 kB of archives.
After this operation, 361 kB of additional disk space will be used.
Get:1 http://kali.download/kali kali-rolling/main amd64 vsftpd amd64 3.0.3-13 [155 kB]
Fetched 155 kB in 1s (204 kB/s)
Preconfiguring packages ...

78Selecting previously unselected package vsftpd.
(Reading database ... 314172 files and directories currently installed.)
Preparing to unpack .../vsftpd_3.0.3-13_amd64.deb ...
7Progress: [  0%] [..........................................................] 87Progress: [ 20%] [###########...............................................] 8

In [10]:
# configure FTP Server
! man vsftpd.conf

VSFTPD.CONF(5)                File Formats Manual               VSFTPD.CONF(5)

NNAAMMEE
       vsftpd.conf - config file for vsftpd

DDEESSCCRRIIPPTTIIOONN
       vsftpd.conf  may  be used to control various aspects of vsftpd's behav‐
       iour. By default, vsftpd looks for this file at the  location  //eettcc//vvss‐‐
       ffttppdd..ccoonnff.  However, you may override this by specifying a command line
       argument to vsftpd. The command line argument is the  pathname  of  the
       configuration file for vsftpd. This behaviour is useful because you may
       wish to use an advanced inetd such as xxiinneettdd to launch vsftpd with dif‐
       ferent configuration files on a per virtual host basis.

FFOORRMMAATT
       The format of vsftpd.conf is very simple. Each line is either a comment
       or a directive. Comment lines start with a # and are ignored. A  direc‐
       tive line has the format:

       option=value

       It  i

### Configure and restart FTP Server
- open /etc/vsftpd.conf to change settings
- allow users in /etc/passwd to login
    - local_enable=YES
- allow uploading to the FTP server
    - write_enable=YES
- bypass writable check; add the following directive
    - allow_writable_chroot=YES
- files are served by default from /srv/ftp
- restart FTP server
```bash
sudo service vsftpd restart
```

- add a new user say ftp-user

```
$ sudo adduser ftp-user
```

- make /home/ftp-user not writable

```
$ sudo chmod a-w /home/ftp-user
```

In [28]:
# restart FTP server after modifying the conf settings
! echo kali | sudo -S systemctl restart vsftpd

[sudo] password for kali: 

## install ftp client if missing

In [ ]:
! echo kali | sudo -S apt install ftp -y

In [ ]:
# run update if ftp install fails
! echo kali | sudo -S apt update

In [11]:
! echo kali | sudo -S apt install tcpdump

Reading package lists... Done
Building dependency tree       
Reading state information... Done
tcpdump is already the newest version (4.9.3-6).
tcpdump set to manually installed.
The following packages were automatically installed and are no longer required:
  libcdio18 libmpdec2 libprotobuf22 libx264-155 libx264-159
Use 'sudo apt autoremove' to remove them.
0 upgraded, 0 newly installed, 0 to remove and 0 not upgraded.


In [12]:
! echo kali | sudo -S apt install dsniff

Reading package lists... Done
Building dependency tree       
Reading state information... Done
dsniff is already the newest version (2.4b1+debian-29).
The following packages were automatically installed and are no longer required:
  libcdio18 libmpdec2 libprotobuf22 libx264-155 libx264-159
Use 'sudo apt autoremove' to remove them.
0 upgraded, 0 newly installed, 0 to remove and 0 not upgraded.


## Sniff user accounts in plain text
- ftp, telnet, pop3, etc.
- run ftp server on a VM and try to connect to it from a a different VM
```bash
ftp <server ip>
<username>
<password>
```

In [ ]:
# run tcpdump from a terminal
# tcpdump -l -X 'ip host <ip>'
! echo kali | sudo tcpdump -l -X 'ip host 192.168.1.148'

In [ ]:
# run dsniff from a terminal
! dsniff -n
# displays ftp username:password captured only after clinet disconnects!

## Install and run telnet server
- not secure; use ssh
- https://www.cyberciti.biz/faq/howto-install-enable-telnet-on-debian-linux/

In [11]:
! echo kali | sudo -S apt install telnet telnetd -y

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
telnet is already the newest version (0.17-42).
The following packages were automatically installed and are no longer required:
  libomp-9-dev libomp5-9 linux-image-5.9.0-kali4-amd64 python3-plotly
Use 'sudo apt autoremove' to remove them.
The following NEW packages will be installed:
  telnetd
0 upgraded, 1 newly installed, 0 to remove and 535 not upgraded.
Need to get 45.4 kB of archives.
After this operation, 114 kB of additional disk space will be used.
Get:1 http://kali.download/kali kali-rolling/main amd64 telnetd amd64 0.17-42 [45.4 kB]
Fetched 45.4 kB in 2s (26.3 kB/s)

78Selecting previously unselected package telnetd.
(Reading database ... 314273 files and directories currently installed.)
Preparing to unpack .../telnetd_0.17-42_amd64.deb ...
7Progress: [  0%] [..........................................................] 87Progress: [ 20%] [###########.........................

In [12]:
! echo kali | sudo -S /etc/init.d/xinetd restart  

### Install and configure telnet server on Ubuntu
- follow directions from: http://jdav.is/2018/04/22/installing-and-enabling-telnet-server-on-ubuntu-linux/
- if you edit /etc/xinetd.d/telnet config file, updated the following line:

`
server = /usr/sbin/telnetd
`

# Active sniffing
- in a switched network environment, packets are only sent to the port they are destined for (using MAC addresses)
- promiscuous devices aren't able to sniff any additional packets that aren't destined for them
- a switch usually has multiple ethernet ports with its own MAC addresses

## arp spoofing

- arp spoofing allows the attacker's to redirect the victims packets to her system
- send a bunch of bogus ARP reply with attackers MAC but victim's IP; saying hey this is my IP and my MAC is this...
- victim then refreshes its ARP cache table (unless its explicitly marked as permanent) or adds new entry
- systems will accept ARP reply even if they don't send out ARP request
- also called ARP cache poisoning
- typically attackers would want to spoof gateway/router/switch that routes all the traffic from a local network out to the Internet

<img src="resources/arp-spoofing.png" />

## steps
    - we'll do a simple MitM between two victims for ethical reasons given we have permission from the victims

1. add the MAC addresses of the victims on the attackers machine

In [15]:
# find victims in the network; find the network address
! echo kali | sudo -S ifconfig

[sudo] password for kali: eth0: flags=4419<UP,BROADCAST,RUNNING,PROMISC,MULTICAST>  mtu 1500
        inet 192.168.195.157  netmask 255.255.255.0  broadcast 192.168.195.255
        inet6 fe80::20c:29ff:febb:4e8a  prefixlen 64  scopeid 0x20<link>
        ether 00:0c:29:bb:4e:8a  txqueuelen 1000  (Ethernet)
        RX packets 231808  bytes 193206092 (184.2 MiB)
        RX errors 0  dropped 0  overruns 0  frame 0
        TX packets 91655  bytes 8957474 (8.5 MiB)
        TX errors 0  dropped 0 overruns 0  carrier 0  collisions 0

lo: flags=73<UP,LOOPBACK,RUNNING>  mtu 65536
        inet 127.0.0.1  netmask 255.0.0.0
        inet6 ::1  prefixlen 128  scopeid 0x10<host>
        loop  txqueuelen 1000  (Local Loopback)
        RX packets 88364  bytes 62522967 (59.6 MiB)
        RX errors 0  dropped 0  overruns 0  frame 0
        TX packets 88364  bytes 62522967 (59.6 MiB)
        TX errors 0  dropped 0 overruns 0  carrier 0  collisions 0



In [20]:
# run nmap ping scan (disable port scan) to find hosts in the network
! nmap -sn 192.168.195.0/24

Starting Nmap 7.80 ( https://nmap.org ) at 2020-09-30 00:58 EDT
Nmap scan report for 192.168.195.1
Host is up (0.0011s latency).
Nmap scan report for 192.168.195.2
Host is up (0.00092s latency).
Nmap scan report for 192.168.195.131
Host is up (0.0031s latency).
Nmap scan report for 192.168.195.140
Host is up (0.0057s latency).
Nmap scan report for 192.168.195.157
Host is up (0.0018s latency).
Nmap scan report for 192.168.195.160
Host is up (0.0014s latency).
Nmap done: 256 IP addresses (6 hosts up) scanned in 2.45 seconds


- See victims ARP cache
```bash
arp -n
arp -d <ip address> # delete entry; you;ll see incomplete entry
```

In [ ]:
# ping a victim ip
! ping -c 1 -w 1 <victim1 IP>

In [ ]:
# ping another vitim ip
! ping -c 1 -w 1 <victim2 IP>

In [ ]:
# check the atackers arp cache
! arp -n

2. enable ip forwarding capability in the kernel so the victims do get their packets and are not suspicious of any spoofing activities

In [25]:
# check ip_forward settings - 0 means disabled
! echo kali | sudo -S cat /proc/sys/net/ipv4/ip_forward

[sudo] password for kali: 1


In [ ]:
! echo kali | sudo -S sysctl net.ipv4.ip_forward

In [26]:
# enable if necessary on the fly
! echo kali | sudo -S sysctl -w net.ipv4.ip_forward=1

[sudo] password for kali: net.ipv4.ip_forward = 1


### enable ip forwarding permanently
- open /etc/sysctl.conf file in an editor add the following line
```
net.ipv4.ip_forward = 1
```
- run the following command to enable the changes made

```
$ sysctl -p /etc/sysctl.conf
```

- or on Ubuntu run the following command

```
$ /etc/init.d/procps.sh restart
```

3. run arpspoof program to sniff packets on a switched LAN

In [27]:
! man arpspoof

ARPSPOOF(8)                 System Manager's Manual                ARPSPOOF(8)

NNAAMMEE
       arpspoof - intercept packets on a switched LAN

SSYYNNOOPPSSIISS
       aarrppssppooooff [--ii _i_n_t_e_r_f_a_c_e] [--cc _o_w_n_|_h_o_s_t_|_b_o_t_h] [--tt _t_a_r_g_e_t] [--rr] _h_o_s_t

DDEESSCCRRIIPPTTIIOONN
       aarrppssppooooff redirects packets from a target host (or all hosts) on the LAN
       intended for another host on the LAN by forging ARP replies.   This  is
       an extremely effective way of sniffing traffic on a switch.

       Kernel  IP  forwarding  (or  a  userland program which accomplishes the
       same, e.g. fragrouter(8)) must be turned on ahead of time.

OOPPTTIIOONNSS
       --ii _i_n_t_e_r_f_a_c_e
              Specify the interface to use.

       --cc _o_w_n_|_h_o_s_t_|_b_o_t_h
              Specify which hardware address t use when restoring the arp

#### run arpspoof
- must run as super user/root

```
$ sudo arpspoof -i <interface> -t <targetIP or victim1IP> -r <hostIP or victim2IP>

```
- let the arpspoof continue to run
- run dsniff on another terminal to sniff username and password
- make a victim connect to FTP server running on another victim
```bash
ftp <victimIP Running FTP server>
<username>
<password>
```
- close the connection and you'll see the username and password on dsniff terminal

## Wireshark

- use wireshark to caputure the sniffed traffic

- https://www.wireshark.org/docs/wsug_html_chunked/ChapterIntroduction.html
- https://www.wireshark.org/docs/wsug_html_chunked/ChWorkBuildDisplayFilterSection.html
- some filters - see manage saved bookmarks for exampes (Ribbon on the left side of filter bar)
- add/remove filters for quick reference: Analyze -> Display Filters 

## Use FTP server to capture account info and data
- configure FTP
- common FTP commands
```bash
ftp <ip of FTP server>
<username>
<password>
ls
cd <folderName>
put filename.txt # upload a file
get filename.txt # download a file
```

## Some common filters

```
ip.src == 192.168.1. and ip.dst == 192.168.1.122
ip.src == <clientIP> and ip.dst == <ftp server IP> and tcp.port == 21
```
    
### check data contents on TCP traffic with keyword
```  
tcp contains PASS 
tcp contains secret
frame contains <word>
```

### Use Regular Expression

- match SSN format
```
tcp matches "[0-9]{3}\-[0-9]{2}\-[0-9]{4}"
```
    
- match 16 digit credit card numbers with spaces in between
```
tcp matches "[0-9]{4} [0-9]{4} [0-9]{4} [0-9]{4}"
```